In [1]:
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [2]:
import pandas as pd

dfEmbeddings = pd.read_csv(
    'C:/Users/alire/OneDrive/data/statman_bitbucket/aki/LLM/March2024/openai_3large_operation.csv'
)
dfEmbeddings.head()

,project_id,operation_no,X1,X2,X3,X4,X5,X6,X7,X8,...,X3063,X3064,X3065,X3066,X3067,X3068,X3069,X3070,X3071,X3072
0,PR-00000001,1,0.011898,0.036309,-0.005463,0.008832,0.014760,0.027967,-0.035566,0.017884,...,-0.010715,0.010800,-0.012413,0.012261,0.008452,0.008917,0.010977,-0.029892,-0.010766,-0.010437
1,PR-00000002,2,0.000332,0.027386,-0.020735,-0.030093,0.011078,0.034497,-0.051592,-0.001025,...,-0.004109,0.002861,0.003123,-0.000474,0.001826,0.023946,0.022539,-0.020903,-0.003339,0.002045
2,PR-00000003,3,-0.013588,0.006824,-0.002654,0.027271,0.033145,0.019402,-0.031663,0.027783,...,0.004871,0.006225,-0.000025,0.002941,-0.011971,0.019294,0.002917,0.003523,0.004716,0.002019
3,PR-00000004,4,0.028444,0.027613,-0.017272,-0.006343,0.022245,0.052718,-0.014736,0.047054,...,0.011954,-0.006590,0.000384,-0.010179,0.002074,0.015004,0.001219,-0.010087,0.011813,0.037503
4,PR-00000005,5,0.015713,0.057943,-0.016102,0.009510,-0.026413,0.027086,-0.050581,0.033581,...,-0.001399,-0.003857,-0.012810,-0.008986,-0.001552,0.009922,0.005754,-0.013910,-0.013042,-0.006614


In [3]:
dfPatients = pd.read_csv(
    'C:/Users/alire/OneDrive/data/statman_bitbucket/aki/LLM/March2024/patients_for_python.csv'
)
dfPatients.head()

,project_id,operation_no,is_female,age,ethnicity,height,weight,broad_diagnosis_grouping,transformed_specific_procedure,count_diagnoses,...,kdigo_stage_creat,stay_duration_days,survival_days,urine_output_ml_kg_below_0.5,urine_output_ml_kg_below_0.3,anuria,kdigo_stage_urine,urine_output_ml_kg_avg,optime,kdigo_stage
0,PR-00000001,1,1,18.11,white,148.0,80.9,NaN,NaN,2,...,0.0,3.0,NaN,9.0,5.0,0.0,1.0,0.993370,112.0,0.0
1,PR-00000002,2,0,17.29,white,179.0,77.5,NaN,NaN,3,...,NaN,5.0,NaN,2.0,0.0,0.0,0.0,1.811853,87.0,NaN
2,PR-00000003,3,0,17.18,white,180.0,65.0,NaN,NaN,2,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PR-00000004,4,1,18.23,BAME,169.0,56.1,NaN,NaN,3,...,0.0,25.0,NaN,16.0,5.0,0.0,2.0,1.061992,144.0,1.0
4,PR-00000005,5,1,16.86,white,166.0,61.6,NaN,NaN,3,...,0.0,11.0,NaN,4.0,0.0,0.0,0.0,0.910138,114.0,0.0


In [8]:
dfPatients_subset = dfPatients.loc[:, ['project_id', 'operation_no', 'kdigo_stage']].dropna()
dfPatients_subset.head()

,project_id,operation_no,kdigo_stage
0,PR-00000001,1,0.0
3,PR-00000004,4,1.0
4,PR-00000005,5,0.0
5,PR-00000006,6,0.0
6,PR-00000007,7,0.0


In [11]:
dfCombined = pd.merge(
    dfPatients_subset
    , dfEmbeddings
    , on = ['project_id', 'operation_no']
    , how = 'inner'
)

In [16]:
X, y = dfCombined.iloc[:, 3:].to_numpy(), dfCombined.iloc[:, 2].to_numpy()

In [18]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [93]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
import copy

class FeatureExtractor_Classifier:
    def __init__(self, **kwargs):
        self.knn = KNeighborsClassifier(**kwargs)
        return None
    
    def fit(self, X, y, cv = 5):

        if not isinstance(cv, int):
            raise TypeError("'cv' must be an integer")
        
        # create folds
        kf = KFold(n_splits = cv)
        kf.get_n_splits(X)
        self.kfolds = kf
        self.nfolds = cv
        
        # train model within each fold
        trained_models = []
        insample_prediction_proba = np.empty(len(y), dtype = float)
        for i, (train_index, test_index) in enumerate(kf.split(X)):
            tmp_knn = copy.deepcopy(self.knn)
            tmp_knn.fit(X[train_index, :], y[train_index])
            insample_prediction_proba[test_index] = tmp_knn.predict_proba(X[test_index, :])[:, 1]
            trained_models.append(tmp_knn)

        self.trained_models = trained_models
        self.insample_prediction_proba = insample_prediction_proba
        return self
    
    def predict_proba(self, X = None):

        if X is None:
            return self.insample_prediction_proba
        
        all_preds = np.empty((X.shape[0], self.nfolds), dtype = float)
        for n in range(self.nfolds):
            all_preds[:, n] = self.trained_models[n].predict_proba(X)[:, 1]
        return np.mean(all_preds, axis = 1)

class FeatureExtractor_Regressor:
    def __init__(self, cv = 5):
        pass
    
    def fit(self, X, y):
        pass
    
    def predict(self, X):
        pass


In [97]:
my_fe = FeatureExtractor_Classifier(n_neighbors = 50).fit(X = X_train, y = y_train, cv = 10)

In [99]:
my_fe.predict_proba(X_test)

array([0.324, 0.272, 0.364, 0.168, 0.264, 0.334, 0.318, 0.22 , 0.224,
       0.32 , 0.212, 0.464, 0.284, 0.33 , 0.126, 0.204, 0.344, 0.388,
       0.284, 0.224, 0.302, 0.306, 0.44 , 0.272, 0.37 , 0.308, 0.388,
       0.37 , 0.426, 0.366, 0.306, 0.17 , 0.456, 0.36 , 0.298, 0.388,
       0.126, 0.436, 0.388, 0.186, 0.36 , 0.454, 0.266, 0.388, 0.37 ,
       0.338, 0.184, 0.302, 0.394, 0.126, 0.37 , 0.184, 0.29 , 0.606,
       0.466, 0.314, 0.168, 0.482, 0.306, 0.244, 0.18 , 0.574, 0.344,
       0.236, 0.554, 0.198, 0.306, 0.31 , 0.28 , 0.36 , 0.392, 0.376,
       0.172, 0.3  , 0.316, 0.236, 0.308, 0.22 , 0.32 , 0.254, 0.24 ,
       0.438, 0.36 , 0.144, 0.408, 0.33 , 0.456, 0.456, 0.234, 0.352,
       0.168, 0.37 , 0.302, 0.278, 0.504, 0.396, 0.516, 0.342, 0.334,
       0.36 , 0.302, 0.31 , 0.224, 0.264, 0.21 , 0.216, 0.126, 0.302,
       0.232, 0.288, 0.44 , 0.252, 0.36 , 0.126, 0.272, 0.126, 0.388,
       0.434, 0.276, 0.278, 0.282, 0.384, 0.198, 0.332, 0.184, 0.18 ,
       0.334, 0.36 ,

In [69]:
x = np.empty(10, dtype = float)

In [70]:
x[:2] = 1

In [71]:
x

array([1.000e+000, 1.000e+000, 9.881e-324, 1.482e-323, 1.976e-323,
       4.195e-321, 4.200e-321, 4.204e-321, 4.209e-321, 4.214e-321])